# **Recitation 12**

# Reference Code
Consider the following reference code for gradient descent on logistic regression from homework.

In [ ]:
# Import packages
import numpy as np

In [ ]:
# Generate Logistic Regression Data

# Constants
tau = 1 # Standard deviation of each class
n   = 40  # Examples per class
trunc = 5 # Number of initial steps to remove while plotting

# Generate red and green centers in [0,1]x[0,1] box
np.random.seed(1)
r = np.array([1,0])
g = np.array([-1,0])

# Generate red and green points
rpoints = (np.random.randn(n,2) * tau) + r
gpoints = (np.random.randn(n,2) * tau) + g

# Create data
X = np.vstack([rpoints, gpoints])
Y = np.append(np.ones(n), np.zeros(n))

In [ ]:
# Gradient Descent Code

# Gradient Desent Parameters
w0  = np.array([1,1]) # Initial w
t0  = 1     # Initial step-size
eps = 1e-5  # Stoping criteria

# sigmoid function
def sig(z):
  return 1/(1+np.exp(-z))
sig = np.vectorize(sig) # Allows us to apply this to entire arrays

# *negative* log-likelihood (since we are maximizing)
def nll(w):
  # Inputs to the sigmoid.
  wX = w @ X.T

  # Vector of log probabilities
  # Note: "*" is element-wise multiplication in Python
  logP = Y * np.log(sig(wX)) + (1 - Y) * np.log(sig(-wX))
  return -sum(logP)

# gradient of *negative* log-likelihood (since we are maximizing)
def gradnll(w):
  # Inputs to the sigmoid.
  wX = w @ X.T

  # Calculate the gradient
  # Note = "axis=0" means "sum the rows"
  return -np.sum((Y - sig(wX))[:,np.newaxis] * X, axis=0)

# Define function that performs one iteration of gradient descent, with linesearch
def GDiter(w):
  obj = nll(w) # objective
  grad = gradnll(w) # gradient
  grad_norm2 = np.linalg.norm(grad)**2 # squared norm of gradient

  done = False # Have not completed linesearch
  t = t0 # Set initial step size

  oracle_calls = 0
  # Perform line search
  while not done:
    w_new = w - t * grad

    oracle_calls += 1
    if nll(w_new) > obj - (t/2) * grad_norm2: # If there is enough decrease
      t = 0.5 * t # Shrink step-size by 0.5
    else:
      done = True

  # Return new point
  return w_new, oracle_calls

# Perform gradient descent
i = 0 # Counts iterations
w = w0
grad_norm = np.linalg.norm(gradnll(w))

# Run gradient descent
i = 0
oracle_calls = 0
while grad_norm > eps: # Stopping condition
  i += 1
  w, calls = GDiter(w)
  oracle_calls += calls # Count oracle calls

  grad_norm = np.linalg.norm(gradnll(w))
  obj = nll(w)

  # Print progress
  print('iter = %s; oracle_calls = %s; w = %s; neg-loglik=%.4g; ||grad|| = %.3g'%(i, oracle_calls, w, obj, grad_norm))

# **(a)**
In the line-search described by the lecture-notes, if the condition $f\left(x^{+}\right)>f(x)-\frac{t}{2}\|\nabla f(x)\|^{2}$ is not met, then we decrease the step-size by a shrikage factor $1/\gamma$. $gamma=2$ is most commonly used, but the $0.5$-factor shrinkage is not sacred. You can also use other factors.

Do the following:

1. Copy-and-paste the reference code.
2. Identify the part where the line-search is done.
3. Change the shrinkage-factor to 0.25.
4. Rerun the code.
5. Comment on what changes you see in behavior of gradient descent.
6. Repeat steps 1-5 with a shrinkage factor of 0.75.


In [ ]:
## ADD CODE FOR PART (a) BELOW ##

# 0.25-shrink version

In [ ]:
# 0.75-shrink version

# **(b)**

**Background:** In this problem, we will compute the "analytic center" of a polyhedron. In particular, suppose we have some vectors $a_1,\dots,a_m \in \mathbb{R}^n$. From the lectures on linear programs, we know that the following inequalities will create a polyhedral region in $n$-dimensional space:

$$a_{i}^{T} x \leq 1, \quad i=1, \ldots, m, \quad\left|x_{i}\right| \leq 1, \quad i=1, \ldots, n$$

Solving for the "analytic center" mathematically formalizes the task of finding a "point in the middle" of this polyhedron. This is done by solving the optimization problem below. For concreteness, we will consider $n=5$ and $m=3$ in this problem.

**Problem:** Given three vectors $a_j \in \mathbb{R}^{5}$ for $j=1,2,3$.
Consider the below, consider the objective function with inputs $x \in \mathbb{R}^{5}$, where we denote $i$-th entry of $x$ with $x_i$. The minimizer of this objective function is the "analytic center" of $a_j$ for $j=1,2,3$.
$$f(x) = -\sum_{i=1}^{5} \ln(1-x_i^2) - \sum_{j=1}^{3}\ln(1 - a_j^T x).$$
Note that the objective is not-every defined. In particular, if $(1-x_i^2)\leq 0$ for any $i$ or if $(1-a_j^T x) \leq 0$ for any $j$, the logarithms are not defined.

Assume our initial point $x_0$ does not run into this issue. There is a way to modify the line-search to prevent future steps from becoming undefined as well.

Specifically, the linesearch from the lecture notes changes to the following:

**IF** $(f\left(x^{+}\right)>f(x)-\frac{t}{2}\|\nabla f(x)\|^{2})$ **OR** (There is $i$ where $(1-x_i^2)\leq 0$) **OR** (There is $j$ where $(1-a_j^T x)\leq 0$):  

  $\quad t = 0.5t$

**ELSE**

  $\quad \texttt{done}$


Do the following:

1. Generate an initial iterate $x_0 = (0,\dots,0)$, i.e. the length-5 vector with all zeros, using `x0=np.zeros(5)`.
2. Set the random seed using `np.random.seed(0)`.
3. Generate the three $a_j$ randomly (i.e. for $j = 1,2,3$) using the `np.random.randn(5)` command.
4. Implement gradient descent on the new $f(x)$ with the line-search described above.

  Hint 1: You could use the reference code as a template.

  Hint 2: The *partial derivative* of $f$ w.r.t. each $x_i$ is $$ \frac{\partial f}{\partial x_i}(x) = \frac{2 x_i}{1-x_i^2} + \sum_{j=1}^3 \frac{a_{ji}}{1-a_j^T x},$$
  where $a_{ji}$ is the $i$-th entry of the vector $a_j$. Use this fact to create the gradient $\nabla f(x)$.
3. Run the code.

In [ ]:
## ADD CODE FOR PART (b) BELOW ##

# **(c)**

Do the following:
1. Copy-and-paste your code from part (b).
2. Change the step-size shrinkage to 0.25.
3. Run the code.
4. Comment on what differences you see in behavior of the algorithm.
5. Repeat steps 1-4 using a step-size shrinkage of 0.75.

In [ ]:
## ADD CODE FOR PART (c) BELOW ##

# 0.25-shrink version

In [ ]:
# 0.75-factor version